In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
truncatedModelPath = "truncatedModel"

truncatedTokenizer = AutoTokenizer.from_pretrained(truncatedModelPath)


fullMT5Name = "google/mt5-small"

fullModel = AutoModelForSeq2SeqLM.from_pretrained(fullMT5Name)
fullTokenizer = AutoTokenizer.from_pretrained(fullMT5Name)

In [6]:
keepTokens = truncatedTokenizer.convert_ids_to_tokens([i for i in range(truncatedTokenizer.vocab_size)])
keepIdInFull = fullTokenizer.convert_tokens_to_ids(keepTokens)

from copy import deepcopy
truncatedModel = deepcopy(fullModel)
truncatedModel.config.vocab_size=len(keepTokens)

In [8]:
from torch import nn

truncatedModel.encoder.embed_tokens = nn.Embedding(len(keepTokens), truncatedModel.config.hidden_size)
truncatedModel.encoder.embed_tokens.data = fullModel.encoder.embed_tokens.weight[keepIdInFull].detach().numpy()

truncatedModel.shared = truncatedModel.encoder.embed_tokens

truncatedModel.decoder.embed_tokens = nn.Embedding(len(keepTokens), truncatedModel.config.hidden_size)
truncatedModel.decoder.embed_tokens.data = fullModel.decoder.embed_tokens.weight[keepIdInFull].detach().numpy()

truncatedModel.shared = truncatedModel.decoder.embed_tokens

truncatedModel.lm_head = nn.Linear(truncatedModel.config.hidden_size, truncatedModel.config.vocab_size, bias=False)
truncatedModel.lm_head.weight.data = fullModel.lm_head.weight.data[keepIdInFull]

In [ ]:
print(truncatedModel)

In [10]:
truncatedModel.save_pretrained(truncatedModelPath)